<a href="https://colab.research.google.com/github/gagan3012/DetectingBias/blob/master/DetectingBias.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime → "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)


Fri Feb 26 18:10:47 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.39       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   77C    P0    35W /  70W |   8492MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [22]:
!pip install tensorflow==1.15 #CPU
!pip install tensorflow-gpu==1.15  # GPU
!pip install -q gpt-2-simple
import gpt_2_simple as gpt2
from datetime import datetime
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [23]:
gpt2.download_gpt2(model_name="124M")

gpt2.mount_gdrive()

Fetching checkpoint: 1.05Mit [00:00, 207Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:00, 3.50Mit/s]
Fetching hparams.json: 1.05Mit [00:00, 586Mit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 498Mit [00:11, 43.5Mit/s]                                  
Fetching model.ckpt.index: 1.05Mit [00:00, 442Mit/s]                                                
Fetching model.ckpt.meta: 1.05Mit [00:00, 4.60Mit/s]
Fetching vocab.bpe: 1.05Mit [00:00, 5.02Mit/s]

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
!pip install -U -q kaggle
!mkdir -p .kaggle

In [5]:
access_key = {"username":"gagan2000","key":"73e5d9db85677b98fe016b25873727ec"}

with open('/root/.kaggle/kaggle.json', 'w') as file:
    json.dump(access_key, file)

In [6]:
!chmod 600 ~/.kaggle/kaggle.json


In [10]:
!kaggle datasets download -d snapcrack/all-the-news -p /content

 94% 229M/244M [00:02<00:00, 91.5MB/s]
100% 244M/244M [00:02<00:00, 91.0MB/s]


In [11]:
!unzip /content/all-the-news.zip -d /content/

Archive:  /content/all-the-news.zip
  inflating: /content/articles1.csv  
  inflating: /content/articles2.csv  
  inflating: /content/articles3.csv  


In [24]:
d1 = pd.read_csv('articles1.csv', encoding = 'utf-8')
d2 = pd.read_csv('articles2.csv', encoding = 'utf-8')
d3 = pd.read_csv('articles3.csv', encoding = 'utf-8')
data = pd.concat([d1, d2, d3])

In [8]:
data.head()

,Unnamed: 0,id,title,publication,author,date,year,month,url,content
42566,146028,218078,An eavesdropping Uber driver saved his 16-year...,Washington Post,Avi Selk,2016-12-30,2016.0,12.0,https://web.archive.org/web/20161231004909/htt...,Uber driver Keith Avila picked up a p...
42567,146029,218079,Plane carrying six people returning from a Cav...,Washington Post,Sarah Larimer,2016-12-30,2016.0,12.0,https://web.archive.org/web/20161231004909/htt...,Crews on Friday continued to search L...
42568,146030,218080,After helping a fraction of homeowners expecte...,Washington Post,Renae Merle,2016-12-30,2016.0,12.0,https://web.archive.org/web/20161231004909/htt...,When the Obama administration announced a...
42569,146031,218081,"Yes, this is real: Michigan just banned bannin...",Washington Post,Chelsea Harvey,2016-12-30,2016.0,12.0,https://web.archive.org/web/20161231004909/htt...,This story has been updated. A new law in...
42570,146032,218082,What happened in Washington state after voters...,Washington Post,Christopher Ingraham,2016-12-29,2016.0,12.0,https://web.archive.org/web/20161231004909/htt...,The nation’s first recreational marijuana...


In [9]:
data['content'] = data['content'].fillna("")
data['title'] = data['title'].fillna("")
content = data['content']
titles = data["title"]

In [10]:
final_right = data[data['publication'].str.contains("Fox")]
final_right = final_right[['title','content']]
final_right.shape

(4354, 2)

In [11]:
final_left = data[data['publication'].str.contains("CNN")]
final_left = final_left[['title','content']]
final_left.shape

(11488, 2)

In [14]:
final_right['content'].to_csv("rightcontent.txt",header=None, index=None, sep='\t', mode='a')
final_left['content'].to_csv("leftcontent.txt",header=None, index=None, sep='\t', mode='a')

In [15]:
sess = gpt2.start_tf_sess()
file_name_1 = "rightcontent.txt"

In [16]:
#for right content
gpt2.finetune(sess,
              dataset=file_name_1,
              model_name='124M',
              steps=1000,
              restore_from='fresh',
              run_name='run_right_1',
              print_every=10,
              sample_every=200,
              save_every=500,
              )

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Loading checkpoint models/124M/model.ckpt
INFO:tensorflow:Restoring parameters from models/124M/model.ckpt


  0%|          | 0/1 [00:00<?, ?it/s]

Loading dataset...


100%|██████████| 1/1 [00:27<00:00, 27.54s/it]


dataset has 6117244 tokens
Training...
[10 | 28.12] loss=2.88 avg=2.88
[20 | 50.07] loss=2.94 avg=2.91
[30 | 72.59] loss=2.86 avg=2.89
[40 | 95.85] loss=2.91 avg=2.90
[50 | 120.19] loss=2.76 avg=2.87
[60 | 144.08] loss=2.61 avg=2.82
[70 | 167.67] loss=2.74 avg=2.81
[80 | 191.56] loss=2.85 avg=2.81
[90 | 215.56] loss=2.83 avg=2.82
[100 | 239.47] loss=2.67 avg=2.80
[110 | 263.38] loss=2.75 avg=2.80
[120 | 287.36] loss=2.68 avg=2.79
[130 | 311.39] loss=2.83 avg=2.79
[140 | 335.42] loss=2.91 avg=2.80
[150 | 359.48] loss=2.93 avg=2.81
[160 | 383.57] loss=2.52 avg=2.79
[170 | 407.59] loss=2.78 avg=2.79
[180 | 431.57] loss=2.73 avg=2.78
[190 | 455.56] loss=2.73 avg=2.78
[200 | 479.51] loss=2.59 avg=2.77
======== SAMPLE 1 ========
 cities or the states through which they live. According to Politico, he has been trying to make it work for Republicans in general:<|endoftext|>This is the official website for the NBA    “””’ games. The NBA NBA Live website features a live game feed with the latest

In [18]:
gpt2.generate(sess, run_name='run_right_1',
              length=100,
              temperature=.7,
              nsamples=10,
              batch_size=10,
              prefix="Allowing more immigrants to come into the United States will",
              truncate="<|endoftext|>",
              include_prefix=False
              )

Allowing more immigrants to come into the United States will help create jobs, while allowing them to stay here in the hope that they will be able to find work while they are in the U. S.’s free youth labor market. ”                                                            
Allowing more immigrants to come into the United States will help us all.   But instead of helping the countless Americans who have been victims of crime, let me make clear: we will be victimized by one more criminal generation.                                                                 
Allowing more immigrants to come into the United States will help us all,” he said. ”We all have a responsibility to help them, but it will only benefit us if they assimilate into our country.” Trump also said he would not accept more Muslim immigrants. ”When I say, ’I am going to deport all Muslims,’ I mean, that’s not going to happen,” Trump said, as he addressed the crowd at the University of Phoenix. ”I want to be very
Al

In [20]:
gpt2.copy_checkpoint_to_gdrive(run_name='run_right_1')

In [ ]:
sess = gpt2.start_tf_sess()
file_name_2 = "leftcontent.txt"

In [19]:
#for left content
gpt2.finetune(sess,
              dataset=file_name_2,
              model_name='124M',
              steps=1000,
              restore_from='fresh',
              run_name='run_left_1',
              print_every=10,
              sample_every=200,
              save_every=500,
              )

ValueError: ignored

In [ ]:
gpt2.generate(sess, run_name='run_left_1',
              length=100,
              temperature=.7,
              nsamples=10,
              batch_size=10,
              prefix="Allowing more immigrants to come into the United States will",
              truncate="<|endoftext|>",
              include_prefix=True
              )